## Load and Preprocess Data Manually

Since we are not using `ImageDataGenerator`, you need to load and preprocess images manually.

In [1]:
import os
import cv2
import numpy as np

# Define dataset path
DATASET_PATH = "datasets/asl_alphabet_train/asl_alphabet_train"
CLASSES = sorted(os.listdir(DATASET_PATH))  # 29 classes (A-Z, space, del, nothing)

# Image dimensions
IMG_SIZE = 64  # Resize to 64x64 pixels

# Lists to store images and labels
X, Y = [], []

# Load images and labels
for label, class_name in enumerate(CLASSES):
    class_path = os.path.join(DATASET_PATH, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)  # Read image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize
        X.append(img)
        Y.append(label)  # Assign label to image

# Convert lists to NumPy arrays
X = np.array(X, dtype=np.float32) / 255.0  # Normalize images
Y = np.array(Y, dtype=np.int64)  # Labels


KeyboardInterrupt: 

✅ Now, X contains images, and Y contains corresponding labels (0 to 28).

### Split Data Into Training & Validation Sets

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset (80% training, 20% validation)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")


### Build a Custom CNN Without TensorFlow

We'll use NumPy to manually implement convolutions, but for efficiency, I'll use PyTorch (optional) to handle automatic differentiation.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train).permute(0, 3, 1, 2)  # Change shape to (N, C, H, W)
Y_train_tensor = torch.tensor(Y_train)

X_val_tensor = torch.tensor(X_val).permute(0, 3, 1, 2)
Y_val_tensor = torch.tensor(Y_val)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define CNN Model (Custom Implementation)
class SignLanguageCNN(nn.Module):
    def __init__(self):
        super(SignLanguageCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Adjust based on image size
        self.fc2 = nn.Linear(128, 29)  # 29 output classes (A-Z, space, del, nothing)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # No softmax (CrossEntropyLoss applies it)
        return x

# Initialize model, loss function, and optimizer
model = SignLanguageCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


✅Now, we have a PyTorch-based CNN instead of TensorFlow.